# **Practical assignment for Topic 7**

**Answer the following questions by modifying and running the code given in the lecture.**

### Question 1
According to the model `nb`, which we trained using scikit-learn library in the lecture on the email spam dataset, what is the probability of an email being a spam if the email contains just the word "click" 10 times without any other words present? And what if the email contains the word "click" 20 times? Explain the results.

*write your answer here*

### Question 2
Now try the same with our own implementation given in the last cell of the notebook. You will get results that are different from those you got in question 1, which means that something is incorrect. What is the modification necessary in our code to get correct results?

*write your answer here*

### Question 3
With the email spam dataset, compute macro-averaged F1-score of the model `nb` (the one which we trained using scikit-learn library) evaluated using Leave-One-Out Cross-Validation (do not implement LOOCV yourself, just use scikit-learn functionality). What F1-score did you get? Now do the same but with Multinomial Naive Bayes. What F1-score did you get now? Which of the two algorithms would you recommend for this data? Give at least two distinct reasons.

*write your answer here*

### Question 4
Now go to the top of the notebook file from the lecture where we worked with the Iris dataset. Try replacing the model `nb` we used with Multinomial Naive Bayes (with appropriate import) and try to train the model. Do not modify any other code. You will get an error message. Explain the reasons behind the message specifically in the context of Naive Bayes.

*write your answer here*